In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from scipy.stats import skew, kurtosis, median_absolute_deviation
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
import matplotlib.pyplot as plt
data_dir = '../../../data/'

In [24]:
train = pd.read_csv('train.csv').dropna()

In [38]:
# Drop NaN
train = train.dropna()

# Drop outlier
train = train.loc[train['log_return1_realized_volatility'] < 0.02]

In [26]:
print(train.isna().sum().sum())
train.head()

0


,stock_id,time_id,target,row_id,wap1_sum,wap1_mean,wap1_std,wap1_amax,wap1_median_absolute_deviation,wap1_skew,wap1_kurtosis,wap2_sum,wap2_mean,wap2_std,wap2_amax,wap2_median_absolute_deviation,wap2_skew,wap2_kurtosis,wap3_sum,wap3_mean,wap3_std,wap3_amax,wap3_median_absolute_deviation,wap3_skew,wap3_kurtosis,log_return1_sum,log_return1_realized_volatility,log_return1_mean,log_return1_std,log_return1_amax,log_return1_median_absolute_deviation,log_return1_skew,log_return1_kurtosis,log_return2_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_amax,log_return2_median_absolute_deviation,log_return2_skew,log_return2_kurtosis,log_return3_sum,log_return3_realized_volatility,log_return3_mean,log_return3_std,log_return3_amax,log_return3_median_absolute_deviation,log_return3_skew,log_return3_kurtosis,wap_balance_sum,wap_balance_mean,wap_balance_std,price_spread_sum,price_spread_mean,price_spread_std,bid_spread_sum,bid_spread_mean,bid_spread_std,ask_spread_sum,ask_spread_mean,ask_spread_std,ask_price1_sum,ask_price1_mean,ask_price1_std,ask_price1_amax,ask_price1_median_absolute_deviation,ask_price1_skew,ask_price1_kurtosis,ask_price2_sum,ask_price2_mean,ask_price2_std,ask_price2_amax,ask_price2_median_absolute_deviation,ask_price2_skew,ask_price2_kurtosis,bid_price1_sum,bid_price1_mean,bid_price1_std,bid_price1_median_absolute_deviation,bid_price1_skew,bid_price1_kurtosis,bid_price2_sum,bid_price2_mean,bid_price2_std,bid_price2_median_absolute_deviation,bid_price2_skew,bid_price2_kurtosis,total_volume_sum,total_volume_mean,total_volume_std,volume_imbalance_sum,volume_imbalance_mean,volume_imbalance_std,today_by_sma5ratio_sum,today_by_sma5ratio_realized_volatility,today_by_sma5ratio_mean,today_by_sma5ratio_std,today_by_sma10ratio_sum,today_by_sma10ratio_realized_volatility,today_by_sma10ratio_mean,today_by_sma10ratio_std,today_by_sma15ratio_sum,today_by_sma15ratio_realized_volatility,today_by_sma15ratio_mean,today_by_sma15ratio_std,wap1_sum_450,wap1_mean_450,wap1_std_450,wap1_amax_450,wap1_median_absolute_deviation_450,wap1_skew_450,wap1_kurtosis_450,wap2_sum_450,wap2_mean_450,wap2_std_450,wap2_amax_450,wap2_median_absolute_deviation_450,wap2_skew_450,wap2_kurtosis_450,wap3_sum_450,wap3_mean_450,wap3_std_450,wap3_amax_450,wap3_median_absolute_deviation_450,wap3_skew_450,wap3_kurtosis_450,log_return1_sum_450,log_return1_realized_volatility_450,log_return1_mean_450,log_return1_std_450,log_return1_amax_450,log_return1_median_absolute_deviation_450,log_return1_skew_450,log_return1_kurtosis_450,log_return2_sum_450,log_return2_realized_volatility_450,log_return2_mean_450,log_return2_std_450,log_return2_amax_450,log_return2_median_absolute_deviation_450,log_return2_skew_450,log_return2_kurtosis_450,log_return3_sum_450,log_return3_realized_volatility_450,log_return3_mean_450,log_return3_std_450,log_return3_amax_450,log_return3_median_absolute_deviation_450,log_return3_skew_450,log_return3_kurtosis_450,...,bid_price2_median_absolute_deviation_50,bid_price2_skew_50,bid_price2_kurtosis_50,total_volume_sum_50,total_volume_mean_50,total_volume_std_50,volume_imbalance_sum_50,volume_imbalance_mean_50,volume_imbalance_std_50,today_by_sma5ratio_sum_50,today_by_sma5ratio_realized_volatility_50,today_by_sma5ratio_mean_50,today_by_sma5ratio_std_50,today_by_sma10ratio_sum_50,today_by_sma10ratio_realized_volatility_50,today_by_sma10ratio_mean_50,today_by_sma10ratio_std_50,today_by_sma15ratio_sum_50,today_by_sma15ratio_realized_volatility_50,today_by_sma15ratio_mean_50,today_by_sma15ratio_std_50,wap1_sum_25,wap1_mean_25,wap1_std_25,wap1_amax_25,wap1_median_absolute_deviation_25,wap1_skew_25,wap1_kurtosis_25,wap2_sum_25,wap2_mean_25,wap2_std_25,wap2_amax_25,wap2_median_absolute_deviation_25,wap2_skew_25,wap2_kurtosis_25,wap3_sum_25,wap3_mean_25,wap3_std_25,wap3_amax_25,wap3_median_absolute_deviation_25,wap3_skew_25,wap3_kurtosis_25,log_return1_sum_25,log_return1_realized_volatility_25,log_return1_mean_25,log_return1_std_25,log_retu

In [44]:
select_features_df = pd.read_csv('RemoveOutliers_200_good_features.csv')
select_features = select_features_df['Feature'].values.tolist()

In [45]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def train_and_evaluate(train):
    # Hyperparammeters (just basic)
    params = {
      'objective': 'rmse',  
      'boosting_type': 'gbdt',
      'num_leaves': 100,
      'n_jobs': 6,
      'learning_rate': 0.1,
      'feature_fraction': 0.8,
      'bagging_fraction': 0.8,
      'verbose': -1
    }
    

    # Split features and target
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    x = x[select_features]
    y = train['target']
    
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    
    # Create a KFold object
    kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)
    feature_importance = pd.DataFrame()
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 10000, 
                          early_stopping_rounds = 50, 
                          verbose_eval = 50,
                          feval = feval_rmspe)
        
        # Get Feature_importance
        importances = pd.DataFrame({f'Feature': model.feature_name(), 
                                f'Fold{fold+1}_Importance': model.feature_importance(importance_type='gain')})
        if len(feature_importance) == 0:
            feature_importance = importances
        else:
            feature_importance = feature_importance.merge(importances, on=['Feature'], how='right')
        
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')

    
#     importances = pd.DataFrame({'Feature': model.feature_name(), 
#                                 'Importance': model.feature_importance(importance_type='gain')})
#     importances.sort_values(by = 'Importance', inplace=True)
  
    
#     importances2 = importances.nlargest(100,'Importance', keep='first').sort_values(by='Importance', ascending=True)
#     print(importances2)
#     list_sample = importances2['Feature'].to_list()
#     print(list_sample)
# #     importances2.to_csv('Feature.csv',index = False)

#     importances4 = importances.nlargest(100,'Importance', keep='first').sort_values(by='Importance')
#     print('特徴量弱い順')
#     print(importances4)
    
    
    
#     importances3 = importances.nlargest(50,'Importance', keep='first').sort_values(by='Importance', ascending=True)
#     list_sample = importances3['Feature'].to_list()
#     print(list_sample)
    
    
#     importances3[['Importance', 'Feature']].plot(kind = 'barh', x = 'Feature', figsize = (8,6), color = 'blue', fontsize=11);plt.ylabel('Feature', fontsize=12)
        # Return test predictions
    return feature_importance


df_imp = train_and_evaluate(train)

Training fold 1
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000465	training's RMSPE: 0.215376	valid_1's rmse: 0.000491267	valid_1's RMSPE: 0.227527
[100]	training's rmse: 0.000447444	training's RMSPE: 0.207245	valid_1's rmse: 0.000487183	valid_1's RMSPE: 0.225636
[150]	training's rmse: 0.00043545	training's RMSPE: 0.201689	valid_1's rmse: 0.00048719	valid_1's RMSPE: 0.225639
Early stopping, best iteration is:
[107]	training's rmse: 0.000445489	training's RMSPE: 0.206339	valid_1's rmse: 0.000486905	valid_1's RMSPE: 0.225507
Training fold 2
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000464523	training's RMSPE: 0.215174	valid_1's rmse: 0.000495992	valid_1's RMSPE: 0.229636
[100]	training's rmse: 0.000446831	training's RMSPE: 0.206979	valid_1's rmse: 0.000492314	valid_1's RMSPE: 0.227933
[150]	training's rmse: 0.000434991	training's RMSPE: 0.201494	valid_1's rmse: 0.000491936	valid_1's RMSPE: 0.227758
[200]	

In [40]:
# Baseline : 0.22879566032525933
# Good Feature 100 : 0.22749466853138792
# Good Feature 200 : 0.2276787203027581

# Remove Outlier Baseline : 0.2274554248493092
# Good Feature 100 : 0.22675797359161529
# Good Feature 200 : 0.22694096355165244

df_imp['sum'] = 0
for i in range(1, 6):
    df_imp['sum'] += df_imp[f'Fold{i}_Importance']

df_imp_sum = df_imp.sort_values(by=['sum'], ascending=False)
df_imp_sum[:200].to_csv('RemoveOutliers_200_good_features.csv')
df_imp_sum[:100].to_csv('RemoveOutliers_100_good_features.csv')
df_imp_sum.head()

,Feature,Fold1_Importance,Fold2_Importance,Fold3_Importance,Fold4_Importance,Fold5_Importance,sum
23,log_return1_realized_volatility,145559.612779,164779.493345,252301.607541,224032.847270,220865.179779,1.007539e+06
629,log_return1_realized_volatility_25,106078.963518,87183.294791,45277.361593,13436.407384,34089.934866,2.860660e+05
326,log_return1_realized_volatility_150,21250.963915,40792.302301,25358.611670,31010.206468,37131.817144,1.555439e+05
697,today_by_sma5ratio_realized_volatility_25,41.679403,47297.238309,220.891116,38.519006,46922.908237,9.452124e+04
701,today_by_sma10ratio_realized_volatility_25,46501.222224,36.744323,248.317885,37936.617911,533.496329,8.525640e+04


In [29]:
df_imp[:300].to_csv('300_good_features.csv')